In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -U pip
!pip install -U setuptools wheel
!pip install autogluon

  Using cached autogluon-1.4.0-py3-none-any.whl.metadata (11 kB)
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 168.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 96.6 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 68.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824

In [ ]:
import pandas as pd
import numpy as np
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/dacon/elec2025/data/

/content/drive/MyDrive/Colab Notebooks/dacon/elec2025/data


In [ ]:
future_covariates = [
    'temp', 'prec', 'wind', 'hum', 'isolation',
    'sunshine', 'use', 'area_1', 'area_2', 'hour', 'month', 'holiday',
    'sin_hour', 'cos_hour', 'sin_date', 'cos_date',
    'sin_weekday', 'cos_weekday', 'summer_sin', 'summer_cos', 'temp_F',
    'temp2', 'THI', 'WC', 'is_rain', 'log_temp', 'wind_power', 'dew_point',
    'solar_per_hour', 'CDH',
    'target_mean_1', 'target_std_1', 'target_min_1', 'target_max_1', 'z_score',
    'min_temp', 'max_temp', 'min_wind', 'max_wind',
    'min_hum', 'max_hum', 'mean_THI', 'mean_CDH', 'min_log_temp', 'max_log_temp', 'mean_WC'
]

train_df = pd.read_csv('train_p_final2.csv')
train_df["log_power"] = np.log(train_df["power"])
cat_features = ['holiday','use']
for col in cat_features:
    train_df[col] = train_df[col].astype('category')
num_cols = train_df.select_dtypes(include=["number"]).columns
train_df[num_cols] = train_df[num_cols].clip(lower=-1e6, upper=1e6)
train_df.rename(columns={'build_num':'item_id', 'date_time':'timestamp'}, inplace=True)
train_df.drop(columns=['num_date_time'], inplace=True)
train_df.drop(columns=drop_col, inplace=True)

data = TimeSeriesDataFrame(train_df)
predictor = TimeSeriesPredictor(
    prediction_length=7*24,
    target="power",
    eval_metric="SMAPE",
    known_covariates_names=future_covariates,
)
# seed 고정
predictor.fit(data, random_seed=42, presets='best_quality', num_val_windows=7*24)
predictor.refit_full()

Beginning AutoGluon training...
AutoGluon will save models to '/content/drive/MyDrive/Colab Notebooks/dacon/elec2025/data/AutogluonModels/ag-20250821_020107'
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          12
GPU Count:          1
Memory Avail:       80.81 GB / 83.48 GB (96.8%)
Disk Space Avail:   181.02 GB / 235.68 GB (76.8%)
Setting presets to: best_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': SMAPE,
 'hyperparameters': 'default',
 'known_covariates_names': ['temp',
                            'prec',
                            'wind',
                            'hum',
                            'isolation',
                            'sunshine',
                            'use',
                            'area_1',
                            'area

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/821M [00:00<?, ?B/s]

	-0.0690       = Validation score (-SMAPE)
	42.77   s     = Training runtime
	4.20    s     = Validation (prediction) runtime
Training timeseries model ChronosFineTuned[bolt_small]. 


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/191M [00:00<?, ?B/s]

	Saving fine-tuned model to /content/drive/MyDrive/Colab Notebooks/dacon/elec2025/data/AutogluonModels/ag-20250821_020107/models/ChronosFineTuned[bolt_small]/W0/fine-tuned-ckpt
	Saving fine-tuned model to /content/drive/MyDrive/Colab Notebooks/dacon/elec2025/data/AutogluonModels/ag-20250821_020107/models/ChronosFineTuned[bolt_small]/W1/fine-tuned-ckpt
	Saving fine-tuned model to /content/drive/MyDrive/Colab Notebooks/dacon/elec2025/data/AutogluonModels/ag-20250821_020107/models/ChronosFineTuned[bolt_small]/W2/fine-tuned-ckpt
	Saving fine-tuned model to /content/drive/MyDrive/Colab Notebooks/dacon/elec2025/data/AutogluonModels/ag-20250821_020107/models/ChronosFineTuned[bolt_small]/W3/fine-tuned-ckpt
	Saving fine-tuned model to /content/drive/MyDrive/Colab Notebooks/dacon/elec2025/data/AutogluonModels/ag-20250821_020107/models/ChronosFineTuned[bolt_small]/W4/fine-tuned-ckpt
	Saving fine-tuned model to /content/drive/MyDrive/Colab Notebooks/dacon/elec2025/data/AutogluonModels/ag-20250821_

{'SeasonalNaive': 'SeasonalNaive_FULL',
 'RecursiveTabular': 'RecursiveTabular_FULL',
 'DirectTabular': 'DirectTabular_FULL',
 'NPTS': 'NPTS_FULL',
 'DynamicOptimizedTheta': 'DynamicOptimizedTheta_FULL',
 'AutoETS': 'AutoETS_FULL',
 'ChronosZeroShot[bolt_base]': 'ChronosZeroShot[bolt_base]_FULL',
 'ChronosFineTuned[bolt_small]': 'ChronosFineTuned[bolt_small]_FULL',
 'TemporalFusionTransformer': 'TemporalFusionTransformer_FULL',
 'DeepAR': 'DeepAR_FULL',
 'PatchTST': 'PatchTST_FULL',
 'TiDE': 'TiDE_FULL',
 'WeightedEnsemble': 'WeightedEnsemble_FULL'}

In [ ]:
history_df = pd.read_csv('train_p_final2.csv')
history_df["log_power"] = np.log(history_df["power"])
cat_features = ['holiday','use']
for col in cat_features:
    history_df[col] = history_df[col].astype('category')
num_cols = history_df.select_dtypes(include=["number"]).columns
history_df[num_cols] = history_df[num_cols].clip(lower=-1e6, upper=1e6)
history_df.rename(columns={'build_num':'item_id', 'date_time':'timestamp'}, inplace=True)
history_df.drop(columns=['num_date_time'], inplace=True)

# TimeSeriesDataFrame 변환
history_tsd = TimeSeriesDataFrame(history_df)

# 미래 covariates 준비
test_df = pd.read_csv('./test_p_final2.csv')
test_df.drop(columns=['num_date_time'], inplace=True)
for col in ['holiday','use']:
    test_df[col] = test_df[col].astype('category')
num_cols = test_df.select_dtypes(include=["number"]).columns
test_df[num_cols] = test_df[num_cols].clip(lower=-1e6, upper=1e6)
test_df.rename(columns={'build_num':'item_id', 'date_time':'timestamp'}, inplace=True)

future_cov_tsd = TimeSeriesDataFrame(test_df[['item_id','timestamp'] + future_covariates])

# 예측
pred = predictor.predict(
    data=history_tsd,            # 과거 power 포함된 데이터
    known_covariates=future_cov_tsd
)

Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble_FULL


In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission['answer'] = pred.reset_index()['mean']
submission.to_csv('./autogluon_timeseries_feature_all.csv', index=False)

In [ ]:
model_list = [
 'ChronosFineTuned[bolt_small]_FULL',
 'ChronosZeroShot[bolt_base]_FULL',
 'DirectTabular_FULL',
 'TiDE_FULL',
 'RecursiveTabular_FULL',
 'NPTS_FULL',
 'SeasonalNaive_FULL',
 'DynamicOptimizedTheta_FULL',
 'PatchTST_FULL',
 'AutoETS_FULL',
 'TemporalFusionTransformer_FULL',
 'DeepAR_FULL',
]

history_df = pd.read_csv('train_p_final.csv')
history_df["log_power"] = np.log(history_df["power"])
cat_features = ['holiday','use']
for col in cat_features:
    history_df[col] = history_df[col].astype('category')
num_cols = history_df.select_dtypes(include=["number"]).columns
history_df[num_cols] = history_df[num_cols].clip(lower=-1e6, upper=1e6)
history_df.rename(columns={'build_num':'item_id', 'date_time':'timestamp'}, inplace=True)
history_df.drop(columns=['num_date_time'], inplace=True)

# TimeSeriesDataFrame 변환
history_tsd = TimeSeriesDataFrame(history_df)

# 미래 covariates 준비
test_df = pd.read_csv('./test_p_final.csv')
test_df.drop(columns=['num_date_time'], inplace=True)
for col in ['holiday','use']:
    test_df[col] = test_df[col].astype('category')
num_cols = test_df.select_dtypes(include=["number"]).columns
test_df[num_cols] = test_df[num_cols].clip(lower=-1e6, upper=1e6)
test_df.rename(columns={'build_num':'item_id', 'date_time':'timestamp'}, inplace=True)

future_cov_tsd = TimeSeriesDataFrame(test_df[['item_id','timestamp'] + future_covariates])


for model in model_list:
    # 예측
    pred = predictor.predict(
        data=history_tsd,            # 과거 power 포함된 데이터
        known_covariates=future_cov_tsd,
        model=model
    )
    submission = pd.read_csv('sample_submission.csv')
    submission['answer'] = pred.reset_index()['mean']
    submission.to_csv(f'./autogluon_{model}_feature_all.csv', index=False)